# Move from OLD to BIDs structure
Need to specify the origin filenames and the destination filenames.

In [3]:
#IMPORTS
import os
from tqdm import tqdm
from shutil import copyfile

In [4]:
OLD_DATA_PATH = "/media/german/DATA/msrims/data/"
NEW_DATA_PATH = "/media/german/DATA/msrims/bids/"

# ORIGINAL
filenames = ("T2star_fixed.nii.gz", "flair_original.nii.gz", "FLAIRstar.nii.gz", "phase.nii.gz", "mp2rage.nii.gz")#, "qsm_EPI_ph.json", "qsm_EPI_ph.nii.gz")
filenames_to = ("acq-mag_T2star.nii.gz", "FLAIR.nii.gz", "acq-star_FLAIR.nii.gz", "acq-phase_T2star.nii.gz", "acq-MP2RAGEuni_T1map.nii.gz")#, "acq-QSM_T2star.json", "acq-QSM_T2star.nii.gz")
#filenames_ignore = ("T2star.nii.gz", "FLAIR_ce-star.nii.gz", "T2star_phase.nii.gz", "T1map_ce-MP2RAGE.nii.gz", "QSM.json", "QSM.nii.gz")

# DERIVATIVES
all_der_titles = ["mask_predictions_rimnet_v1", "rims_annotations", "registrations_to_T2star"]
all_der_filenames = [("maskPredictions.nii.gz", ), ("maskLesions.nii.gz", ), ("flair_original_REG.nii.gz", "mp2rage_REG.nii.gz")]
all_der_filenames_to = [("mask-predictions_rimnet-v1.nii.gz", ), ("mask.nii.gz", ), ("FLAIR_REG.nii.gz", "acq-MP2RAGEuni_T1map_REG.nii.gz")]

In [7]:
OLD_DATA_PATH = "/media/german/Germs_DATA/T1maps/"
NEW_DATA_PATH = "/media/german/Germs_DATA/bids/"

# ORIGINAL
filenames = ["T1map.nii.gz", ]
filenames_to = ["acq-MP2RAGE_T1map.nii.gz", ]

# DERIVATIVES
all_der_titles = []#"expert_annotations", ]
all_der_filenames = []#("rims_on_T1.nii.gz", ), ]
all_der_filenames_to = []#("expertsannotations.nii.gz", ), ]

In [8]:
TO_REPLACE = False
datasets = (list(range(1, 56)), 
            list(range(56, 86)) + list(range(92, 111)) + list(range(112, 119)) + list(range(120, 133)),
           list(range(86, 91)))
datasets_names = ("CHUV_RIM", "BASEL_INSIDER", "BRUSSELS_RIM")
        
# BIDSIGNORE
"""
if not os.path.exists(NEW_DATA_PATH):
        os.makedirs(NEW_DATA_PATH)
with open(os.path.join(NEW_DATA_PATH, ".bidsignore"), "w+") as bignore:
    for to_ignore in filenames_ignore:
        bignore.write("*" + to_ignore+"\n")
"""

# Creating fold structure
for folder in tqdm(next(os.walk(OLD_DATA_PATH))[1]):
    patient_id = int(folder.split('/')[-1])
    dataset = None
    for i in range(len(datasets)):
        if patient_id in datasets[i]:
            dataset = datasets_names[i]
            break
    if dataset is None:
        print(f"Patient {patient_id} skipped")
        continue
    
    # TOP FOLDER
    database_folder = os.path.join(NEW_DATA_PATH, dataset)
    source_folder = os.path.join(database_folder, f"sub-{patient_id:03d}/ses-01/anat")
    if not os.path.exists(source_folder):
        os.makedirs(source_folder)

    for filename, filename_to in zip(filenames, filenames_to):
        origin = os.path.join(os.path.join(OLD_DATA_PATH, folder), filename)
        to = os.path.join(source_folder, f"sub-{patient_id:03d}_ses-01_" + filename_to)
        #print(origin)
        if os.path.exists(origin) and (not os.path.exists(to) or TO_REPLACE):
            copyfile(origin, to)
        
    # CODE FOLDER
    source_folder = os.path.join(database_folder, f"code")
    if not os.path.exists(source_folder):
        os.makedirs(source_folder)
        
    # DERIVATIVE FOLDER
    for der_title, der_filenames, der_filenames_to in zip(all_der_titles, all_der_filenames, all_der_filenames_to):
        derivative_folder = os.path.join(database_folder, f"derivatives/{der_title}/sub-{patient_id:03d}/ses-01")
        if not os.path.exists(derivative_folder):
            os.makedirs(derivative_folder)

        for der_filename, der_filename_to in zip(der_filenames, der_filenames_to):
            origin = os.path.join(os.path.join(OLD_DATA_PATH, folder), der_filename)
            to = os.path.join(derivative_folder,  f"sub-{patient_id:03d}_ses-01_" + der_filename_to)
            if os.path.exists(origin) and (not os.path.exists(to) or TO_REPLACE):
                copyfile(origin, to)
    

100%|██████████| 68/68 [00:03<00:00, 18.94it/s]
